In [1]:
%pwd
%cd /Users/ScottJeen/OneDrive - University of Cambridge/Classes/LM48/Project/L48Project/tti

/Users/ScottJeen/OneDrive - University of Cambridge/Classes/LM48/Project/L48Project/tti


In [2]:
import os

import numpy as np
import pandas as pd
#from tqdm.notebook import trange
#from tqdm import tqdm
import importlib

import tti_explorer
from tti_explorer import config, utils
from tti_explorer.case import simulate_case, CaseFactors
from tti_explorer.contacts import EmpiricalContactsSimulator
from tti_explorer.strategies import TTIFlowModel, RETURN_KEYS

importlib.reload(tti_explorer)
importlib.reload(tti_explorer.config)
importlib.reload(tti_explorer.utils)
importlib.reload(tti_explorer.case)
importlib.reload(tti_explorer.strategies)
importlib.reload(tti_explorer.contacts)

<module 'tti_explorer.contacts' from '/Users/ScottJeen/OneDrive - University of Cambridge/Classes/LM48/Project/L48Project/tti/tti_explorer/contacts.py'>

In [3]:
def print_doc(func):
    print(func.__doc__)

In [4]:
name = 'S3_test_based_TTI'
rng = np.random.RandomState(0)
case_config = config.get_case_config("delve_mostly_uni")
#case_config = config.get_case_config("delve_mostly_old")
#case_config = config.get_case_config("delve_mostly_twenties")
contacts_config = config.get_contacts_config("delve")
policy_config = config.get_strategy_configs("delve", name)[name]
factor_config = utils.get_sub_dictionary(policy_config, config.DELVE_CASE_FACTOR_KEYS)
strategy_config = utils.get_sub_dictionary(policy_config, config.DELVE_STRATEGY_FACTOR_KEYS)

In [5]:
def load_csv(pth):
    return np.loadtxt(pth, dtype=int, skiprows=1, delimiter=",")

In [6]:
path_to_bbc_data = os.path.join("data", "processed")

child_no_school = load_csv(os.path.join(path_to_bbc_data, "childnoschool.csv"))
child_school = load_csv(os.path.join(path_to_bbc_data, "childschool.csv"))
university = load_csv(os.path.join(path_to_bbc_data, "university.csv"))
twenties = load_csv(os.path.join(path_to_bbc_data, "twenties.csv"))
thirties_to_fifties = load_csv(os.path.join(path_to_bbc_data, "thirtiestofifties.csv"))
fifties_to_seventies = load_csv(os.path.join(path_to_bbc_data, "fiftiestoseventies.csv"))
seventy_plus = load_csv(os.path.join(path_to_bbc_data, "seventyplus.csv"))

In [7]:
double_dose = False
vaccine_strategy = 'gov'
#Vaccine strategies are: none, all, gov, young_inc_children, 
#                        young_exc_children

In [8]:
simulate_contacts = EmpiricalContactsSimulator(child_no_school, child_school, university, twenties, thirties_to_fifties, fifties_to_seventies, seventy_plus, double_dose, vaccine_strategy, rng)
tti_model = TTIFlowModel(rng, **strategy_config)

In [9]:
"""n_seperate_starting_cases = 1000
n_repetitions = 10
outputs = list()
death_rates = []

for i in tqdm(range(n_seperate_starting_cases)):
    old_probs = []
    for i in range(n_repetitions):
        case = simulate_case(rng, p_for_categories_continued=old_probs, **case_config)
        case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
        (contacts, old_probs, death_rate) = simulate_contacts(case, double_dose, vaccine_strategy, None, **contacts_config)
        res = tti_model(case, contacts, case_factors)
        outputs.append(res)
        death_rates.append(death_rate)"""

'n_seperate_starting_cases = 1000\nn_repetitions = 10\noutputs = list()\ndeath_rates = []\n\nfor i in tqdm(range(n_seperate_starting_cases)):\n    old_probs = []\n    for i in range(n_repetitions):\n        case = simulate_case(rng, p_for_categories_continued=old_probs, **case_config)\n        case_factors = CaseFactors.simulate_from(rng, case, **factor_config)\n        (contacts, old_probs, death_rate) = simulate_contacts(case, double_dose, vaccine_strategy, None, **contacts_config)\n        res = tti_model(case, contacts, case_factors)\n        outputs.append(res)\n        death_rates.append(death_rate)'

In [10]:
'''to_show = [
    RETURN_KEYS.base_r,
    RETURN_KEYS.reduced_r
]

x = pd.DataFrame(outputs).mean(0).loc[to_show].to_frame().to_numpy().flatten()
base_r = x[0]
effective_r = x[1]
relevant_death_rate_sum = sum(n for n in death_rates if n != -1)
relevant_death_rate_count = sum(1 for n in death_rates if n != -1)
dr = relevant_death_rate_sum / relevant_death_rate_count

print("Average Mortality Rate: " + str(round(dr*100,2)) + "%")
print("Base R: " + str(round(base_r, 3)))
print("Effective R: " + str(round(effective_r, 3)))'''

'to_show = [\n    RETURN_KEYS.base_r,\n    RETURN_KEYS.reduced_r\n]\n\nx = pd.DataFrame(outputs).mean(0).loc[to_show].to_frame().to_numpy().flatten()\nbase_r = x[0]\neffective_r = x[1]\nrelevant_death_rate_sum = sum(n for n in death_rates if n != -1)\nrelevant_death_rate_count = sum(1 for n in death_rates if n != -1)\ndr = relevant_death_rate_sum / relevant_death_rate_count\n\nprint("Average Mortality Rate: " + str(round(dr*100,2)) + "%")\nprint("Base R: " + str(round(base_r, 3)))\nprint("Effective R: " + str(round(effective_r, 3)))'

In [11]:
"""base_r_rates = []
effective_r_rates = []
av_mortality_rates = []
to_show = [
    RETURN_KEYS.base_r,
    RETURN_KEYS.reduced_r
]

for immunity in tqdm(np.linspace(0,1,100)):
    n_seperate_starting_cases = 1000
    n_repetitions = 10
    outputs = list()
    death_rates = []
    
    for i in range(n_seperate_starting_cases):
        old_probs = []
        for i in range(n_repetitions):
            case = simulate_case(rng, p_for_categories_continued=old_probs, **case_config)
            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
            (contacts, old_probs, death_rate) = simulate_contacts(case, None, immunity, None, **contacts_config)
            res = tti_model(case, contacts, case_factors)
            outputs.append(res)
            death_rates.append(death_rate)
            


    x = pd.DataFrame(outputs).mean(0).loc[to_show].to_frame().to_numpy().flatten()
    base_r_rates.append(x[0])
    effective_r_rates.append(x[1])
    relevant_death_rate_sum = sum(n for n in death_rates if n != -1)
    relevant_death_rate_count = sum(1 for n in death_rates if n != -1)
    dr = relevant_death_rate_sum / relevant_death_rate_count
    av_mortality_rates.append(dr)

print(len(base_r_rates))"""

'base_r_rates = []\neffective_r_rates = []\nav_mortality_rates = []\nto_show = [\n    RETURN_KEYS.base_r,\n    RETURN_KEYS.reduced_r\n]\n\nfor immunity in tqdm(np.linspace(0,1,100)):\n    n_seperate_starting_cases = 1000\n    n_repetitions = 10\n    outputs = list()\n    death_rates = []\n    \n    for i in range(n_seperate_starting_cases):\n        old_probs = []\n        for i in range(n_repetitions):\n            case = simulate_case(rng, p_for_categories_continued=old_probs, **case_config)\n            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)\n            (contacts, old_probs, death_rate) = simulate_contacts(case, None, immunity, None, **contacts_config)\n            res = tti_model(case, contacts, case_factors)\n            outputs.append(res)\n            death_rates.append(death_rate)\n            \n\n\n    x = pd.DataFrame(outputs).mean(0).loc[to_show].to_frame().to_numpy().flatten()\n    base_r_rates.append(x[0])\n    effective_r_rates.append(x[1])\

In [ ]:
# vaccine efficacy

vaccine_strategy = 'gov'
#Vaccine strategies are: none, all, gov, young_inc_children, 
#                        young_exc_children

base_r_rates = []
effective_r_rates = []
av_mortality_rates = []
to_show = [
    RETURN_KEYS.base_r,
    RETURN_KEYS.reduced_r
]

for vaccine_efficacy in (np.linspace(0.4,1,100)):
    print(vaccine_efficacy)
    n_seperate_starting_cases = 1000
    n_repetitions = 10
    outputs = list()
    death_rates = []
    
    for i in range(n_seperate_starting_cases):
        old_probs = []
        for i in range(n_repetitions):
            case = simulate_case(rng, p_for_categories_continued=old_probs, **case_config)
            case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
            (contacts, old_probs, death_rate) = simulate_contacts(case, False, vaccine_strategy, vaccine_efficacy, **contacts_config)
            res = tti_model(case, contacts, case_factors)
            outputs.append(res)
            death_rates.append(death_rate)
            


    x = pd.DataFrame(outputs).mean(0).loc[to_show].to_frame().to_numpy().flatten()
    base_r_rates.append(x[0])
    effective_r_rates.append(x[1])
    relevant_death_rate_sum = sum(n for n in death_rates if n != -1)
    relevant_death_rate_count = sum(1 for n in death_rates if n != -1)
    dr = relevant_death_rate_sum / relevant_death_rate_count
    av_mortality_rates.append(dr)

print(len(base_r_rates))

0.4
0.40606060606060607
0.41212121212121217
0.4181818181818182
0.42424242424242425
0.4303030303030303
0.4363636363636364
0.44242424242424244
0.4484848484848485
0.4545454545454546
0.46060606060606063
0.4666666666666667
0.4727272727272728
0.4787878787878788
0.48484848484848486
0.49090909090909096
0.496969696969697
0.503030303030303
0.5090909090909091
0.5151515151515151
0.5212121212121212
0.5272727272727273
0.5333333333333333
0.5393939393939394
0.5454545454545454
0.5515151515151515
0.5575757575757576
0.5636363636363637
0.5696969696969697
0.5757575757575758
0.5818181818181818
0.5878787878787879
0.593939393939394
0.6000000000000001
0.6060606060606061
0.6121212121212122
0.6181818181818182
0.6242424242424243
0.6303030303030304
0.6363636363636364
0.6424242424242425
0.6484848484848484
0.6545454545454545
0.6606060606060606
0.6666666666666667
0.6727272727272727
0.6787878787878788
0.6848484848484848
0.6909090909090909
0.696969696969697
0.7030303030303031
0.7090909090909091
0.7151515151515151
0.721

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
ax1 = plt.subplot(1,2,1)
ax2 = plt.subplot(1,2,2)

x = np.linspace(0.4,1,100)
y1 = [base_r_rates, effective_r_rates]
y2 = [av_mortality_rates]

ax1.plot(x, y1)
ax2.plot(x, y2)
